In [4]:
import time
import importlib
import pandas as pd
rd = importlib.import_module("app.back-end.data")
sp= importlib.import_module("app.back-end.tools.spliceai")

rd.download_selected_database_for_eys_gene("lovd", "", override=False)
rd.download_selected_database_for_eys_gene("gnomad", "", override=False)

lovd_data = rd.parse_lovd()
gnomad_data = rd.parse_gnomad()

rd.set_lovd_dtypes(lovd_data)
rd.set_gnomad_dtypes(gnomad_data)

variants_on_genome = lovd_data["Variants_On_Genome"].copy()

lovd_data = pd.merge(lovd_data["Variants_On_Transcripts"],
                       variants_on_genome[['id', 'VariantOnGenome/DNA', 'VariantOnGenome/DNA/hg38']],
                       on='id',
                       how='left')

gnomad_data = gnomad_data.copy()
data = rd.merge_gnomad_lovd(lovd_data, gnomad_data)
first_100_rows = data.head(100).copy()
fasta_path = "hg38_chr6.fa"
start_time = time.time()
result_data_spliceai = sp.add_spliceai_eval_columns(first_100_rows[50:55], fasta_path)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to add SpliceAI evaluation column: {elapsed_time:.2f} seconds")
result_data_spliceai

The file at ../app/back-end/src/workspace/lovd/lovd_data.txt already exists.
The file at ../app/back-end/src/workspace/gnomad/gnomad_data.csv already exists.
Time taken to add SpliceAI evaluation column: 15.80 seconds


,id,transcriptid,effectid,position_c_start,position_c_start_intron,position_c_end,position_c_end_intron,VariantOnTranscript/DNA,VariantOnTranscript/RNA,VariantOnTranscript/Protein,...,variant_id_gnomad,Delta score (acceptor gain)_spliceai,Delta score (acceptor loss)_spliceai,Delta score (donor gain)_spliceai,Delta score (donor loss)_spliceai,Delta position (acceptor gain)_spliceai,Delta position (acceptor loss)_spliceai,Delta position (donor gain)_spliceai,Delta position (donor loss)_spliceai,Max_Delta_Score_spliceai
0,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720567-T-C,0.0,0.0,0.0,0.0,63720568,63720587,63720568,63720605,0.0
1,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720567-T-G,0.0,0.0,0.0,0.0,63720577,63720522,63720577,63720605,0.0
2,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720569-G-C,0.0,0.0,0.0,0.0,63720568,63720587,63720570,63720577,0.0
3,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720569-G-T,0.0,0.0,0.0,0.0,63720570,63720587,63720547,63720605,0.0
4,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,...,6-63720570-T-C,0.0,0.0,0.0,0.0,63720618,63720587,63720605,63720577,0.0
